In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from utils import *

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

keys = ['采集时间', '水温', 'pH', '溶解氧', '电导率', '浊度', '高锰酸盐指数','氨氮', '总磷', '总氮']
en_keys = ['WaterTemperature', 'PH' ,'dissolved oxygen', 'Conductivity','Turbidity','PermanganateIndex',
        'AmmoniaNitrogen','TP','TN', 'humidity','room temperature','chlorophyll','Algae density']
# limits = [(5, 30), (5.0, 9), (1, 15), (50, 500), (0, 1500), (0, 15), (0, 0.5), (0, 0.3), (0, 5)]

np.set_printoptions(formatter = {'float': '{:.2e}'.format})


In [ ]:
import torch
from trainer import *
from models import *

def test(ckpt_path, model, data, lPre=42, lGet=84, step=3):
    ckpt = torch.load(ckpt_path)
#     ckpt = torch.load(ckpt_path, map_location='cuda:0')

#     model.load_state_dict(ckpt, strict=False)
    model.load_state_dict(ckpt['state_dict'])
    model.freeze()
    
    l, f = data.shape
    data = np.transpose(data)
    pre = np.zeros((l-lGet, f))
    data = torch.from_numpy(data).to(torch.float32)
    data = data[None, ...]
    print(data.shape)
    max_idx = l - step - lGet
    
    i = 0
    while i <= max_idx:
        x = data[:, :, i: i + lGet]
        y = model(x)
        y = y.numpy().squeeze()
        pre[i : i + step] = y[:, :step].transpose()
        i += step
        
    return pre

kwargs = {'features': 9,
        'lPre': 42,
        'lGet' : 84,
        'Tree_levels':2,
        'hidden_size_rate':4,
        'loss':F.l1_loss,
        'lr':9e-3,
         'descaler':None}
    
model = SCIModule(**kwargs)
# model = SCINet(9, 42, 84, 2, 4)
with torch.no_grad():
    pre = test('./lightning_logs/7d/checkpoints/last.ckpt', model, df.values, lPre=42, lGet=84, step=3)

In [ ]:
lPre, lGet = 42, 84
features = data.shape[1]
l, h = 25, 3

window = 8000
index = x[window:]
real = data[window:, :]
prediction = pre[window:, :]

def plot(x, data, pre, area=9000, lGet=84):
    fig, axis = plt.subplots(features, 1, figsize=(l, h*features), constrained_layout=True)

    for i in range(features):
        name = en_keys[i]
        axis[i].plot(x[:], data[:, i], '-k', linewidth=3)
        axis[i].plot(x[lGet:area+lGet], pre[:area, i], '-r', linewidth=0.8)
        axis[i].plot(x[area+lGet:], pre[area:, i], '-b', linewidth=0.8)

    #     df.plot(y=k, ax=axis[i], style='-k')
    #     df.plot(y=f'{k}(pre)', ax=axis[i], style='--r')

        axis[i].set_title(name, fontsize=20)
        axis[i].set_xlabel('', fontsize=15)
        axis[i].set_ylabel('', fontsize=15)

        axis[i].legend([name], fontsize=15)
    
plot(index, real, prediction, 9000-window)

In [ ]:
df = fujiang_factory('./original_data/05-涪江/涪江水质断面水质-小时尺度/元坝子审核数据查询表.xls', 3, 5, 3)

In [3]:
# !!! Do Not remove !!!
# Data Handler for BianJie, FuJiang, luguhu.
dataHandler('./original_data/04-四川省边界断面/四川边界断面水质数据/', 12, 6, './all_data/bianjie_1d/', data_factory, 3, 5, 3)
dataHandler('./original_data/泸沽湖邛海鲁班水库水质数据/原始查询/', 12, 6, './all_data/luguhu_1d/', data_factory, 3, 5, 3)
dataHandler('./original_data/05-涪江/涪江水质断面水质-小时尺度/', 12, 6, './all_data/fujiang_1d//', data_factory, 3, 5, 3)

19it [00:15,  1.20it/s]
4it [00:04,  1.02s/it]
15it [00:09,  1.62it/s]


In [7]:
# !!! Do Not remove !!!
# Data Handler for MinTuoJiang data
import re
from tqdm import tqdm
from utils import _gen_data
lPre, lGet = 6, 12
r, limit, step = 3, 5, 3
save_path = './all_data/mtj_1d/'
ptj_keys = ['监测时间', '水温(℃)', 'pH值(无量纲)', '溶解氧(mg/L)', '电导率(uS/cm)', '浊度(NTU)','高锰酸盐指数(mg/L)',
           '氨氮(mg/L)', '总磷(mg/L)', '总氮(mg/L)']
p = Path('./original_data/08-岷沱江数据/岷沱江水质监测数据/')
d = {}
for file in p.iterdir():
    name = re.match('\d*?\D+', file.stem)[0]
    if name in d.keys():
        d[name].append(file)
    else:
        d[name] = [file]
for k in tqdm(d.keys()):
    dfs = []
    for file in d[k]:
        df = pd.read_excel(file, header=1, usecols=ptj_keys, index_col=0, dtype=str)
        df.drop(df.index[0], axis=0, inplace=True)
        dfs.append(df)    
    df = pd.concat(dfs)
    df = df.sort_index()
    df.index=pd.to_datetime(df.index, format='%Y-%m-%d %H:%M:%S')
    for f in df.keys():
        df[f] = df[f].str.extract('(^\d+\.\d+)')
    df = df.astype('float64')
    df = df.resample('4H').mean()
    df.loc[(df==0).all(axis=1)] = np.nan    
    df = remove_outliers(df, standard_deviation, 25)
    df[df < 0] = np.nan
    df = patch_up(df, r, limit)
    df = smooth(df, step)
    
    save_file_name = f'{save_path}{k}'
    describe_save_name = f'{save_file_name}_describe.csv'
    _gen_data(df, lGet, lPre, save_file_name)
    df.describe().to_csv(describe_save_name)

100%|███████████████████████████████████████████| 50/50 [01:12<00:00,  1.45s/it]


In [16]:
p = Path('./all_data/fujiang_1d/')
for file in p.iterdir():
    name = file.name
    name = name.replace('审', '')
#     name = name.replace('原始查询（', '')
#     name = name.replace('）', '')
    file.rename(f'./all_data/luguhu_1d/{name}')
    